#Copyright and License

In [1]:
#@title ####Dataset and model Copyright 2018 The TensorFlow Authors

In [2]:
#@title ####Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [3]:
#@title ####Answer selection and feedback type developed by the Computational HCI Laboratory, University of Michigan

#Setup

##Libraries

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
import random
import pandas as pd
from google.colab import widgets
import ipywidgets
from IPython.display import display, clear_output
import sys

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib

##Conditions

In [5]:
def random_none():
  print('random+none')

In [6]:
def random_corrective():
  print('random+corrective')

In [7]:
def random_informative():
  print('random+informative')

In [8]:
def counterfactual_none():
  print('counterfactual+none')

In [9]:
def counterfactual_corrective():
  print('counterfactual+corrective')

In [10]:
def counterfactual_informative():
  print('counterfactual+informative')

In [11]:
def own_prev_none():
  print('own_prev_none')

In [12]:
def own_prev_corrective():
  print('own_prev_corrective')

In [13]:
def own_prev_informative():
  print('own_prev_informative')

In [14]:
def other_prev_none():
  print('other_prev+none')

In [15]:
def other_prev_corrective():
  print('other_prev+corrective')

In [16]:
def other_prev_informative():
  print('other_prev+informative')

##Initialize

In [17]:
#@title def set_condition(distractor, feedback)
def set_condition(d, f):
  if (d == 'r'):
    if (f == 'n'):
      random_none()
    elif (f == 'c'):
      random_corrective()
    else:
      random_informative()
  elif (d == 'c'):
    if (f == 'n'):
      counterfactual_none()
    elif (f == 'c'):
      counterfactual_corrective()
    else:
      counterfactual_informative()
  elif (d == 'o1'):
    if (f == 'n'):
      own_prev_none()
    elif (f == 'c'):
      own_prev_corrective()
    else:
      own_prev_informative()
  else:
    if (f == 'n'):
      other_prev_none()
    elif (f == 'c'):
      other_prev_corrective()
    else:
      other_prev_informative()

In [18]:
#@title def initialize()
def initialize():
  distractor_type = input('Select distractor type [r|c|o1|o2]: ')
  if (distractor_type != 'r' and distractor_type != 'c' and distractor_type != 'o1' and distractor_type != 'o2'):
    sys.exit("Invalid distractor type.")

  feedback_type = input('Select feedback type [n|c|i]: ')
  if (feedback_type != 'n' and feedback_type != 'c' and feedback_type != 'i'):
    sys.exit("Invalid feedback type.")

  set_condition(distractor_type, feedback_type)

##Model

In [19]:
def generate_model():
  dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
  data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
  data_dir = pathlib.Path(data_dir)
  list(data_dir.glob('*'))
  roses = list(data_dir.glob('roses/*'))
  sunflowers = list(data_dir.glob('sunflowers/*'))
  dandelions = list(data_dir.glob('dandelion/*'))
  daisys = list(data_dir.glob('daisy/*'))
  tulips = list(data_dir.glob('tulips/*'))
  batch_size = 32
  img_height = 180
  img_width = 180

  train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

  val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

  class_names = train_ds.class_names

  #dataset configuration
  AUTOTUNE = tf.data.AUTOTUNE

  train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
  val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

  normalization_layer = layers.Rescaling(1./255)

  normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
  image_batch, labels_batch = next(iter(normalized_ds))
  first_image = image_batch[0]
  # Notice the pixel values are now in `[0,1]`.
  #print(np.min(first_image), np.max(first_image))

  data_augmentation = keras.Sequential(
    [
      layers.RandomFlip("horizontal",
                        input_shape=(img_height,
                                    img_width,
                                    3)),
      layers.RandomRotation(0.1),
      layers.RandomZoom(0.1),
    ]
  )

  #build keras model
  num_classes = len(class_names)

  model = Sequential([
    data_augmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, name="outputs")
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  #train the model
  epochs = 15
  history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
  )

  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs_range = range(epochs)